In [1]:
from PandaStation import *
from PandaGrasping import *

# Start a single meshcat server instance to use for the remainder of this notebook.
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])

# Let's do all of our imports here, too.
import numpy as np
import ipywidgets
import pydot
import pydrake.all
import os
from IPython.display import display, SVG
import subprocess


import pydrake.all
from pydrake.geometry import Cylinder, Box
from pydrake.all import (
    RigidTransform, RotationMatrix, AngleAxis, RollPitchYaw, InverseKinematics, MultibodyPlant, Parser,
    FindResourceOrThrow, Solve, PiecewisePolynomial, TrajectorySource, SceneGraph, DiagramBuilder,
    AddMultibodyPlantSceneGraph, LinearBushingRollPitchYaw, MathematicalProgram, AutoDiffXd, GenerateHtml, Role,
    LeafSystem, AbstractValue, PublishEvent, TriggerType, BasicVector, PiecewiseQuaternionSlerp, Quaternion,
    RandomGenerator, UniformlyRandomRotationMatrix, BodyIndex
    )
import pydrake.perception as mut
import open3d as o3d
from ompl import base as ob
from ompl import geometric as og
import time
from enum import Enum
from RRT import RRTStar

ycb = {"cracker": "drake/manipulation/models/ycb/sdf/003_cracker_box.sdf", 
    "sugar": "drake/manipulation/models/ycb/sdf/004_sugar_box.sdf", 
    "soup": "drake/manipulation/models/ycb/sdf/005_tomato_soup_can.sdf", 
    "mustard": "drake/manipulation/models/ycb/sdf/006_mustard_bottle.sdf", 
    "gelatin": "drake/manipulation/models/ycb/sdf/009_gelatin_box.sdf", 
    "meat": "drake/manipulation/models/ycb/sdf/010_potted_meat_can.sdf",
    "brick": "drake/examples/manipulation_station/models/061_foam_brick.sdf"}

for key in ycb.keys():
    ycb[key] = FindResourceOrThrow(ycb[key])
    
ycb["puck"] = FindResource("models/puck.urdf")
ycb["marble"] = FindResource("models/marble.urdf")
ycb["dumbbell"] = FindResource("models/dumbbell.urdf")

def ycb_resource(name):
    global ycb
    return ycb[name]

def random_ycb_resource():
    global ycb
    ycb_items = list(ycb.items())
    index = np.random.randint(0, len(ycb_items))
    return ycb_resource(ycb_items[index][0])

In [43]:
body_index = None

class Stage(Enum):
    INIT = 0
    TO_PREPICK = 1
    TO_PICK = 2
    PICKING = 3
    TO_POSTPICK = 4
    TO_INIT = 5
    
def X_WG_to_q(X_WG):
    translation = X_WG.translation()
    rpy = RollPitchYaw(X_WG.rotation())
    rpy = rpy.vector()
    res = np.concatenate((translation, rpy))
    return res
  
class FindGrasp(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        self.nq = 6
        self.omni_grasp_position_output_port = self.DeclareVectorOutputPort(
                                                                    "omni_grasp_position", 
                                                                    BasicVector(self.nq),
                                                                    self.DoCalcOutput)

        self.station = station
        self.station_context = station_context
        self.rng = np.random.default_rng()
        
          
    def DoCalcOutput(self, context, output):
        global body_index
        start_time = time.time()
        ik_station, body_infos = create_welded_station(self.station, self.station_context, omni = True)
        ik_station_context = ik_station.CreateDefaultContext()
        body_index = body_infos[0].body_index 
        plant = ik_station.get_multibody_plant()
        plant_context = plant.GetMyContextFromRoot(ik_station_context)
        body = plant.get_body(body_index)
        X_WO = plant.EvalBodyPoseInWorld(plant_context, body)
        q_nominal = X_WG_to_q(X_WO)
        q_nominal[2] += 0.1
        q_nominal[3:] = [0, np.pi, 0]
        q, cost = grasp_pose(body_infos[0], ik_station, ik_station_context, q_nominal = q_nominal)
        if not np.isfinite(cost):
            print("WARNING: COULD NOT FIND VALID GRASP POSE, TRYING ANYWAYS")
        output.set_value(q)

class Picker(LeafSystem):
    
    def __init__(self, station, station_context):
        LeafSystem.__init__(self)
        self.nq = 6
        self.grasp_position_input_port = self.DeclareVectorInputPort("grasp_position", BasicVector(self.nq))
        self.omni_position_input_port = self.DeclareVectorInputPort("omni_position", 
                BasicVector(self.nq))
        self.omni_position_command_output_port = self.DeclareVectorOutputPort(
                "omni_position_command", BasicVector(self.nq),
                self.DoCalcOutput)
        self.hand_command_output_port = self.DeclareVectorOutputPort("hand_position_command", BasicVector(1),
                                                                    self.CalcHandOutput)
        self.station = station
        self.station_context = station_context
        self.plant = station.get_multibody_plant()
        self.omni = self.plant.GetModelInstanceByName("omni_arm")
        self.hand = self.plant.GetModelInstanceByName("hand")
        self.plant_context = self.plant.GetMyContextFromRoot(self.station_context)
        self.scene_graph = station.get_scene_graph()
        self.scene_graph_context = station.GetSubsystemContext(self.scene_graph, self.station_context)
        self.query_output_port = self.scene_graph.GetOutputPort("query")
        self.q_S = None
        self.q_G = None
        self.q_P = None
        self.q_post= None
        self.omni_traj = None
        self.hand_traj = None
        self.X_Pregrasp = None
        self.X_Grasp = None
        self.stage = Stage.INIT 
        self.setup_time = 0.5
        self.body_index = None
        
    def CalcHandOutput(self, context, output):
        time = context.get_time()
        if (self.hand_traj is None) or (time > self.hand_traj.end_time()):
            output.set_value([0])
        else:
            output.set_value(self.hand_traj.value(time).flatten())
        
    def DoCalcOutput(self, context, output):
        time = context.get_time()
        if time < self.setup_time: # allow time for objects to settle
            self.q_S = self.EvalVectorInput(context, self.omni_position_input_port.get_index()).get_value().copy()
            output.set_value(self.q_S)
            return
        
        if self.stage == Stage.INIT:
            print("INITIALIZATION")
            self.q_G = self.EvalVectorInput(context, self.grasp_position_input_port.get_index()).get_value()
            self.q_P = self.pregrasp(self.q_G)
            print(self.q_P, self.q_G)
            self.omni_traj = self.custom_rrt_star(self.q_S, time, self.q_P, time + 3)
            self.hand_traj = self.first_order_traj([0.0], time, [0.08], time + 3)
            self.stage = Stage.TO_PREPICK
            print("TO PREPICK")
            
        if self.stage == Stage.TO_PREPICK:
            
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_PICK
                print("TO PICK")
                #self.omni_traj = self.default_rrt_trajectory(self.q_P, time, self.q_G, time + 2)
                self.omni_traj = self.first_order_traj(self.q_P, time, self.q_G, time +2)
                self.hand_traj = self.first_order_traj([0.08], time, [0.08], time +2)
                
        if self.stage == Stage.TO_PICK:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.PICKING
                print("PICKING")
                self.hand_traj = self.first_order_traj([0.08], time, [0.0], time +2)
                self.omni_traj = self.first_order_traj(self.q_G, time, self.q_G, time +2)
        
        if self.stage == Stage.PICKING:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_POSTPICK
                print("TO POSTPICK")
                self.q_post = self.q_G.copy()
                self.q_post[2] += 0.1
                self.omni_traj = self.first_order_traj(self.q_G, time, 
                                                          self.q_post, time + 2)
                self.hand_traj = self.first_order_traj([0.0], time, [0.0], time + 3)
                
        if self.stage == Stage.TO_POSTPICK:
            if time <= self.omni_traj.end_time():
                
                output.set_value(self.omni_traj.value(time).flatten())
            else:
                self.stage = Stage.TO_INIT
                print("TO INITIAL POSITION")

                global body_index
                self.body_index = body_index
                self.omni_traj = self.custom_rrt_star(self.q_post, time, 
                                                          self.q_S, time + 3)
                self.hand_traj = self.first_order_traj([0.0], time, [0.0], time + 3)
                 
        if self.stage == Stage.TO_INIT:
            if time <= self.omni_traj.end_time():
                output.set_value(self.omni_traj.value(time).flatten())
                
    def pregrasp(self, q_G):
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        omni = plant.GetModelInstanceByName("omni_arm")
        hand = plant.GetModelInstanceByName("hand")
        
        plant.SetPositions(plant_context, omni, q_G)
        body = plant.GetBodyByName("panda_hand")
        X_WG = plant.EvalBodyPoseInWorld(plant_context, body)
        X_WP = X_WG.multiply(RigidTransform(RotationMatrix(), [0, 0, -0.07]))
        q_P = self.ik(X_WP, warm_start = q_G)
        return q_P
        
                
    def first_order_traj(self, q_start, start_time, q_end, end_time):
        time = np.array([start_time, end_time])
        qs = np.array([q_start, q_end])
        return PiecewisePolynomial.FirstOrderHold(time, qs.T)
    
   
    def custom_rrt_star(self, q_start, start_time, q_goal, goal_time):
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True, 
                                             body_index_to_weld_to_hand = self.body_index)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        omni = plant.GetModelInstanceByName("omni_arm")
        hand = plant.GetModelInstanceByName("hand")
        scene_graph = ik_station.get_scene_graph()
        scene_graph_context = ik_station.GetSubsystemContext(scene_graph, ik_station_context)
        query_output_port = scene_graph.GetOutputPort("query")
        
        def is_colliding(q):
            plant.SetPositions(plant_context, omni, q)
            query_object = query_output_port.Eval(scene_graph_context)
            return query_object.HasCollisions()
        
        
        limits = self.joint_limits()
        print(f"STARTING RRTSTAR:\nstart:\n{q_start}\ngoal:\n{q_goal}")
        rrt = RRTStar(q_start, q_goal, is_colliding, limits)
        plan, costs = rrt.plan()
        
        assert (plan is not None) and (costs is not None), "No RRT solution found"
        
        times = start_time + (goal_time - start_time)*costs
        
        
        
        if len(plan) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                   plan[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                plan[:, 0:self.nq].T)
       
    
    
    def default_rrt_trajectory(self, q_start, start_time, q_goal, goal_time):
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True,
                                             body_index_to_weld_to_hand = self.body_index)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        omni = plant.GetModelInstanceByName("omni_arm")
        hand = plant.GetModelInstanceByName("hand")
        scene_graph = ik_station.get_scene_graph()
        scene_graph_context = ik_station.GetSubsystemContext(scene_graph, ik_station_context)
        query_output_port = scene_graph.GetOutputPort("query")
        
        def is_colliding(q):
            plant.SetPositions(plant_context, omni, q)
            query_object = query_output_port.Eval(scene_graph_context)
            return query_object.HasCollisions()
        
        def isStateValid(state):
            q = self.parse_state(state)
            return not is_colliding(q)
        
        #def getPathLengthObjective(si):
        #    return ob.PathLengthOptimizationObjective(si)

        joint_limits = self.joint_limits()
        #space = ob.SE3StateSpace()
        space = ob.CompoundStateSpace()
        space.addSubspace(ob.RealVectorStateSpace(3), 1)
        space.addSubspace(ob.RealVectorStateSpace(3), 1)
        bounds = ob.RealVectorBounds(3)
        for i in range(3):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])
        space.getSubspace(0).setBounds(bounds)
        bounds = ob.RealVectorBounds(3)
        
        bounds.setLow(-np.pi)
        bounds.setHigh(np.pi)

        space.getSubspace(1).setBounds(bounds)
        
        ss = og.SimpleSetup(space)
        si = ss.getSpaceInformation()
        pdef = ss.getProblemDefinition()
        #pdef.setOptimizationObjective(getPathLengthObjective(si))
        ss.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))
        start = self.q_to_state(space, q_start)
        print("start:", start)
        assert isStateValid(start), "you have supplied an invalid start state RRT"
        goal = self.q_to_state(space, q_goal)
        print("goal:", goal)
        assert isStateValid(goal), "you have supplied an invalid goal state for RRT"
        ss.setStartAndGoalStates(start, goal) 
        solved = ss.solve()
        assert solved
        ss.simplifySolution()
        path = ss.getSolutionPath()
        res = []
        costs = []
        for state in path.getStates():
            q = self.parse_state(state)
            if len(costs) == 0:
                costs.append(0)
            else:
                d = q - res[-1]
                cost = np.sqrt(d.dot(d))
                costs.append(costs[-1] + cost)
            res.append(q)
        
        if (costs[-1] == 0):
            costs[-1] = 1

        qs = np.array(res)
        times = start_time + ((goal_time - start_time)*np.array(costs)/costs[-1])
        if len(qs) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                    qs[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                qs[:, 0:self.nq].T)
    
    def rrt_star_trajectory(self, q_start, start_time, q_goal, goal_time):
        
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True,
                                             body_index_to_weld_to_hand = self.body_index)
        plant = ik_station.get_multibody_plant()
        ik_station_context = ik_station.CreateDefaultContext()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station_context)
        omni = plant.GetModelInstanceByName("omni_arm")
        hand = plant.GetModelInstanceByName("hand")
        scene_graph = ik_station.get_scene_graph()
        scene_graph_context = ik_station.GetSubsystemContext(scene_graph, ik_station_context)
        query_output_port = scene_graph.GetOutputPort("query")
        
        def is_colliding(q):
            plant.SetPositions(plant_context, omni, q)
            query_object = query_output_port.Eval(scene_graph_context)
            return query_object.HasCollisions()
        
        def isStateValid(state):
            q = self.parse_state(state)
            return not is_colliding(q)

        joint_limits = self.joint_limits()
        space = ob.SE3StateSpace()
        #space = ob.CompoundStateSpace()
        #space.addSubspace(ob.RealVectorStateSpace(3), 1)
        #space.addSubspace(ob.RealVectorStateSpace(3), 1)
        bounds = ob.RealVectorBounds(3)
        for i in range(3):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])
        space.getSubspace(0).setBounds(bounds)
        #bounds = ob.RealVectorBounds(3)
        
        #bounds.setLow(-np.pi)
        #ounds.setHigh(np.pi)
        '''
        for i in range(3,6):
            bounds.setLow(i, joint_limits[i][0])
            bounds.setHigh(i, joint_limits[i][1])
        '''
        #space.getSubspace(1).setBounds(bounds)
        si = ob.SpaceInformation(space)
        si.setStateValidityChecker(ob.StateValidityCheckerFn(isStateValid))
        start = self.q_to_state(space, q_start)
        assert isStateValid(start), "you have supplied an invalid start state RRT"
        goal = self.q_to_state(space, q_goal)
        assert isStateValid(goal), "you have supplied an invalid goal state for RRT"
        pdef = ob.ProblemDefinition(si)
        pdef.setStartAndGoalStates(start, goal)
        pdef.setOptimizationObjective(ob.PathLengthOptimizationObjective(si))
        planner = og.RRTstar(si)
        planner.setProblemDefinition(pdef)
        planner.setup()
        solved = planner.solve(10)
        assert solved
        path = pdef.getSolutionPath()
        res = []
        costs = []
        for state in path.getStates():
            q = self.parse_state(state)
            if len(costs) == 0:
                costs.append(0)
            else:
                d = q - res[-1]
                cost = np.sqrt(d.dot(d))
                costs.append(costs[-1] + cost)
            res.append(q)
        
        if (costs[-1] == 0):
            costs[-1] = 1

        qs = np.array(res)
        times = start_time + ((goal_time - start_time)*np.array(costs)/costs[-1])
        if len(qs) < 3:
            return PiecewisePolynomial.FirstOrderHold(times, 
                    qs[:, 0:self.nq].T)
        
        return PiecewisePolynomial.CubicShapePreserving(times,
                qs[:, 0:self.nq].T)
    
    def joint_limits(self):
        joint_inds = self.plant.GetJointIndices(self.omni)[:self.nq]
        joint_limits = []
        for i in joint_inds:
            joint = self.plant.get_joint(i)
            joint_limits.append((joint.position_lower_limits()[0],
                                 joint.position_upper_limits()[0]))
        return joint_limits

    def parse_state(self, state):
        """ parses ompl RealVectorStateSpace::StateType into a numpy array"""
        
        """
        if type(state) == ob.State:
            q = np.array([state[0], state[1], state[2]])
            quat = np.array([
                    state[6], 
                    state[3], 
                    state[4], 
                    state[5]
                ])
        else:
            q = np.array([state.getX(), state.getY(), state.getZ()])
            quat = np.array([
                    state.rotation().w, 
                    state.rotation().x,
                    state.rotation().y,
                    state.rotation().z
                ])
        
        rpy = RollPitchYaw(Quaternion(quat)).vector()
        q = np.concatenate((q, rpy))
        """
        q = []
        if type(state) == ob.State:
            for i in range(6):
                q.append(state[i])
        else:
            for i in range(2):
                for j in range(3):
                    q.append(state[i][j])
        
        return np.array(q)

    def q_to_state(self, space, q):
        """ turns a python list q into an ompl state"""
        
        """
        state = ob.State(space)
        rpy = RollPitchYaw(q[3:])
        quat = rpy.ToQuaternion()
        rot = np.array([quat.x(), quat.y(), quat.z(), quat.w()])
        q = np.concatenate((q[:3], rot))
        #print("q that makes state:", q)
        for i in range(len(q)):
            state[i] = q[i] 
        
        """
        state = ob.State(space)
        for i in range(len(q)):
            state[i] = q[i] 
        
        return state
            
        
    def q_to_X_WG(self, q):
        assert len(q) == self.nq
        
        self.plant.SetPositions(self.plant_context, self.omni, q)   
        body = self.plant.GetBodyByName("panda_hand")
        return self.plant.EvalBodyPoseInWorld(self.plant_context, body)

        
    def ik(self, X_WG, warm_start = None):
        p_tol = np.ones(3)*0.01
        theta_tol = 0.01
        ik_station, _ = create_welded_station(self.station, self.station_context, omni = True)
        plant = ik_station.get_multibody_plant()
        plant_context = ik_station.GetSubsystemContext(plant, ik_station.CreateDefaultContext())
        ik = InverseKinematics(plant, plant_context)
        ik.AddPositionConstraint(
                plant.GetFrameByName("panda_hand"),
                np.zeros(3),
                plant.world_frame(),
                X_WG.translation() - p_tol,
                X_WG.translation() + p_tol)
        ik.AddOrientationConstraint(
                plant.GetFrameByName("panda_hand"),
                RotationMatrix(),
                plant.world_frame(),
                X_WG.rotation(),
                theta_tol)
        ik.AddMinimumDistanceConstraint(0.001, 0.1)
        q = ik.q()
        prog = ik.prog()
        #warm_start = X_WG_to_q(X_WG)
        if warm_start is not None:
            prog.SetInitialGuess(q, warm_start)
        result = Solve(prog)
        if not result.is_success():
            print("failed pregrasp ik, returning best result")
        return result.GetSolution(q)

def random_position():
    return [0.6 + np.random.uniform(-0.05, 0.05), np.random.uniform(-0.1,0.1), 0.2]

def random_orientation():
    rot = RotationMatrix.MakeZRotation(np.random.uniform(-np.pi, np.pi))
    #rot = rot.multiply(RotationMatrix.MakeXRotation(-np.pi/2))
    return rot

In [44]:
builder = pydrake.systems.framework.DiagramBuilder()
station = builder.AddSystem(OmniStation(time_step = 0.001))
station.SetupTableStation()


rs = np.random.RandomState()  # this is for python
generator = RandomGenerator(rs.randint(1000))  # this is for c++
    
item = "soup"
#station.AddModelFromFile(ycb_resource(item), 
#                    RigidTransform(random_orientation(), 
#                                    random_position()))

station.AddModelFromFile(ycb_resource(item),
                         RigidTransform(random_orientation(), random_position()))


plant = station.get_multibody_plant()
omni = station.GetOmni()
station.Finalize()
station_context = station.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(station_context)
scene_graph = station.get_scene_graph()
scene_graph_context = station.GetSubsystemContext(scene_graph, station_context)
start_pose = plant.EvalBodyPoseInWorld(plant_context, plant.GetBodyByName("panda_hand"))
meshcat = pydrake.systems.meshcat_visualizer.ConnectMeshcatVisualizer(builder,
          scene_graph,
          output_port=station.GetOutputPort("query_object"),
          delete_prefix_on_load=True,                                      
          zmq_url=zmq_url)#, role = Role.kProximity)# <- this commented part allows visualization of the collisions
grasp_finder = builder.AddSystem(FindGrasp(station, station_context))
picker = builder.AddSystem(Picker(station, station_context))
builder.Connect(grasp_finder.GetOutputPort("omni_grasp_position"), picker.GetInputPort("grasp_position"))
builder.Connect(station.GetOutputPort("omni_position_measured"), picker.GetInputPort("omni_position"))
builder.Connect(picker.GetOutputPort("omni_position_command"), station.GetInputPort("omni_position"))
builder.Connect(picker.GetOutputPort("hand_position_command"), station.GetInputPort("hand_position"))
diagram = builder.Build()
simulator = pydrake.systems.analysis.Simulator(diagram)
station_context = station.GetMyContextFromRoot(simulator.get_mutable_context())
grasp_finder.station_context = station_context
picker.station_context = station_context
meshcat.start_recording()
simulator.AdvanceTo(12.5)
meshcat.stop_recording()
meshcat.publish_recording()

Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6026...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7026/static/
Connected to meshcat-server.
INITIALIZATION
[ 0.63532126 -0.06324904  0.24147849  0.          3.14159265  1.06929579] [ 0.63532126 -0.06324904  0.18147849  0.          3.14159265  1.06929579]
STARTING RRTSTAR:
start:
[-1.11395609e-07 -1.13923051e-05  4.99998503e-01  1.03327755e-02
  3.14158936e+00  9.99995800e-03]
goal:
[ 0.63532126 -0.06324904  0.24147849  0.          3.14159265  1.06929579]
TO PREPICK
TO PICK
PICKING
TO POSTPICK
TO INITIAL POSITION
STARTING RRTSTAR:
start:
[ 0.63532126 -0.06324904  0.28147849  0.          3.14159265  1.06929579]
goal:
[-1.11395609e-07 -1.13923051e-05  4.99998503e-01  1.03327755e-02
  3.14158936e+00  9.99995800e-03]


In [4]:
"""
mustard
brick
marble
puck
soup
sugar
gelatin
meat
"""
#file = open(item + ".html", "w")
#file.write(meshcat.vis.static_html())
#file.close()

#print(subprocess.check_output(["tar", "vczf", "media/videos/ycb_pick/" + item + ".gz", item + ".html"]))
#print(subprocess.check_output(["trash", item +".html"]))

'\nmustard\nbrick\nmarble\npuck\nsoup\nsugar\ngelatin\nmeat\n'